In [1]:
import json
import sqlite3
import numpy as np
import geoip2.database
from ua_parser import user_agent_parser

In [2]:
LOG_FILE = "results.log"
DB_FILE = "results.db"

## Populate database

In [3]:
conn = sqlite3.connect(DB_FILE)
c = conn.cursor()
c.execute(
    """CREATE TABLE sessions
             (id text, ip text, ua text, cb text, ww integer, wo text)"""
)
c.execute(
    """CREATE TABLE picks
             (id text, ip text, c1 text, c2 text, o text, dm integer, sp integer, cp integer)"""
)

with open(LOG_FILE) as log_file:
    sessions = picks = bad_picks = 0
    for line in log_file:
        line = json.loads(line)
        if line["type"] == "session":
            sessions += 1
            c.execute(
                "INSERT INTO sessions VALUES (?, ?, ?, ?, ?, ?)",
                (
                    line["id"],
                    line["ip"],
                    line["ua"],
                    line["cbtq"],
                    line["ww"],
                    line["wo"],
                ),
            )
        elif line["type"] == "pick":
            picks += 1
            c.execute(
                "INSERT INTO picks VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                (
                    line["id"],
                    line["ip"],
                    line["c1"],
                    line["c2"],
                    line["o"],
                    line["dm"],
                    line["sp"],
                    line["cp"],
                ),
            )
        elif line["type"] in ["badpick", "badmatch"]:
            bad_picks += 1
        else:
            raise ValueError(f"Invalid log entry type!: {line['type']}")

if (
    c.execute("SELECT count(*) FROM sessions").fetchone()[0]
    > c.execute("SELECT count(DISTINCT id) FROM sessions").fetchone()[0]
):
    raise ValueError("There are duplicate sessions!")

ids = [row for row in c.execute("SELECT id FROM sessions")]
pick_counts = []
empty_sessions = 0
for row in ids:
    pick_count = c.execute("SELECT count(*) FROM picks WHERE id = ?", row).fetchone()[0]
    if pick_count < 1:
        c.execute("DELETE FROM sessions WHERE id = ?", row)
        empty_sessions += 1
    else:
        pick_counts.append(pick_count)

session_count = sessions - empty_sessions

conn.commit()
conn.close()

## Session statistics

In [4]:
print("sessions:", sessions)
print("empty sessions:", empty_sessions)
print("good sessions:", session_count)
print("picks:", picks)
print("bad picks:", bad_picks)
print("average picks per session:", np.mean(pick_counts))
print("median picks per session:", np.median(pick_counts))
print("min picks per session:", np.min(pick_counts))
print("max picks per session:", np.max(pick_counts))

sessions: 2386
empty sessions: 176
good sessions: 2210
picks: 22423
bad picks: 77
average picks per session: 10.146153846153846
median picks per session: 7.5
min picks per session: 1
max picks per session: 302


## Picks per set length

Ten-color sets were disabled shortly after the survey was launched, to avoid diluting the responses for the other set lengths. Thus, there are fewer ten-color responses.

In [5]:
conn = sqlite3.connect(DB_FILE)
c = conn.cursor()

In [6]:
for num_colors in [6, 8, 10]:
    print(f"{c.execute('SELECT COUNT(c1) FROM picks WHERE length(c1) = ?', (num_colors * 7 - 1,)).fetchone()[0]:5d} {num_colors:2d}")

10347  6
10371  8
 1705 10


## Self-reported color-vision-deficiency information

Respondees were asked if they had a color-vision deficiency at the start of the survey.

In [7]:
cvd_types = {
    "na": "Not applicable",
    "dta": "Decline to answer",
    "dk": "Don't know",
    "dy": "Deuteranomaly (partial green deficiency) [most common]",
    "py": "Protanomaly (partial red deficiency)",
    "da": "Deuteranopia (complete green deficiency)",
    "pa": "Protanopia (complete red deficiency)",
    "ty": "Tritanomaly (partial blue deficiency) [rare]",
    "ta": "Tritanopia (complete blue deficiency) [rare]",
    "m": "Monochromasy (complete colorblindness) [rare]",
    "o": "Other",
}

In [8]:
total_cvd_count = 0
for t in cvd_types:
    count = c.execute('SELECT COUNT(cb) FROM sessions WHERE cb = ?', (t,)).fetchone()[0]
    print(f'{count:4d}', cvd_types[t])
    if t not in ['na', 'dta', 'dk']:
        total_cvd_count += count
print(f'{total_cvd_count:4d} CVD total: {total_cvd_count / session_count * 100:.1f}%')

2024 Not applicable
  28 Decline to answer
  36 Don't know
  74 Deuteranomaly (partial green deficiency) [most common]
  27 Protanomaly (partial red deficiency)
   3 Deuteranopia (complete green deficiency)
   6 Protanopia (complete red deficiency)
   4 Tritanomaly (partial blue deficiency) [rare]
   2 Tritanopia (complete blue deficiency) [rare]
   3 Monochromasy (complete colorblindness) [rare]
   3 Other
 122 CVD total: 5.5%


## Screen size information

Horizontal screen resolution was recorded for each user session, rounded to the nearest 100px. The screen aspect ratio, landscape or portrait, was also recorded (square was recorded as landscape).

In [9]:
for i in range(1, 41):
    print(f"{c.execute('SELECT COUNT(ww) FROM sessions WHERE ww = ?', (i,)).fetchone()[0]:3d} {i * 100:4d}px")

  0  100px
  0  200px
 34  300px
318  400px
 11  500px
  8  600px
  5  700px
 20  800px
 10  900px
 46 1000px
 37 1100px
 73 1200px
193 1300px
295 1400px
187 1500px
 93 1600px
106 1700px
 35 1800px
593 1900px
 22 2000px
  6 2100px
 10 2200px
  8 2300px
  5 2400px
 10 2500px
 72 2600px
  1 2700px
  3 2800px
  1 2900px
  1 3000px
  1 3100px
  0 3200px
  0 3300px
  2 3400px
  0 3500px
  1 3600px
  0 3700px
  3 3800px
  0 3900px
  0 4000px


In [10]:
for i in ['l', 'p']:
    print(i, f"{c.execute('SELECT COUNT(wo) FROM sessions WHERE wo = ?', (i,)).fetchone()[0]:4d}")

l 1779
p  431


## Country derived from IP address

The last version of the MaxMind GeoLite2 database that was available under a CC BY-SA 4.0 license is used. It's from 2019-12-24, which was in the middle of the survey period.

In [11]:
geoip_reader = geoip2.database.Reader('GeoLite2-Country_20191224.mmdb')

In [12]:
countries = {}
for row in c.execute(f'SELECT ip FROM sessions'):
    cc = geoip_reader.country(row[0]).country.iso_code
    if cc not in countries:
        countries[cc] = 0
    countries[cc] += 1

In [13]:
for i in sorted(countries.items(), key=lambda i: i[1], reverse=True):
    print(f'{i[1]:3d} {i[0]}')

644 US
213 DE
140 GB
129 JP
 85 CA
 67 FR
 60 IT
 57 BR
 57 AU
 51 IN
 43 NL
 38 RU
 34 CN
 33 CH
 30 PL
 30 TR
 27 AT
 27 ES
 22 BE
 19 SE
 19 FI
 18 CZ
 17 HU
 16 NO
 16 AR
 16 MX
 15 ZA
 15 DK
 15 RO
 15 ID
 14 PT
 14 CO
 10 HK
 10 NZ
 10 PE
  9 CL
  9 TW
  8 UA
  8 VN
  8 PK
  8 GR
  7 EE
  6 IE
  6 SG
  6 BY
  6 IR
  6 BG
  5 IL
  5 KR
  5 MY
  5 HR
  5 LV
  4 SI
  4 TH
  4 BD
  4 MA
  4 SK
  4 PH
  4 AE
  3 None
  3 LB
  3 SA
  3 IQ
  2 LT
  2 BA
  2 RS
  2 UZ
  1 CU
  1 RW
  1 MV
  1 SD
  1 NG
  1 KE
  1 LU
  1 RE
  1 JM
  1 BS
  1 MW
  1 LK
  1 GE
  1 NP
  1 KZ
  1 CR
  1 EC
  1 PA
  1 AZ
  1 MD
  1 AL
  1 CY
  1 QA
  1 BB


## Browser information

As derived from user-agent strings

In [14]:
browser = {}
os = {}
device = {}
for row in c.execute(f'SELECT ua FROM sessions'):
    ua = user_agent_parser.Parse(row[0])
    if ua['user_agent']['family'] not in browser:
        browser[ua['user_agent']['family']] = 0
    browser[ua['user_agent']['family']] += 1
    if ua['os']['family'] not in os:
        os[ua['os']['family']] = 0
    os[ua['os']['family']] += 1
    if ua['device']['family'] not in device:
        device[ua['device']['family']] = 0
    device[ua['device']['family']] += 1

In [15]:
for i in sorted(browser.items(), key=lambda i: i[1], reverse=True):
    print(f'{i[1]:4d} {i[0]}')

1275 Chrome
 537 Firefox
 186 Mobile Safari UI/WKWebView
 106 Apple Mail
  47 Android
  26 Firefox Mobile
  15 Chromium
  12 Other
   2 Safari
   1 Dragon
   1 Firefox iOS
   1 ekoobot
   1 Amazon Silk


In [16]:
for i in sorted(os.items(), key=lambda i: i[1], reverse=True):
    print(f'{i[1]:4d} {i[0]}')

 999 Windows
 450 Mac OS X
 212 Android
 199 Linux
 187 iOS
 138 Ubuntu
  13 Fedora
  12 Chrome OS


In [17]:
for i in sorted(device.items(), key=lambda i: i[1], reverse=True):
    print(f'{i[1]:4d} {i[0]}')

1360 Other
 450 Mac
 164 iPhone
  27 Generic Smartphone
  22 iPad
   7 OnePlus ONEPLUS A3003
   6 Pixel
   4 ANE-LX2J
   4 Samsung SM-G955U
   4 Samsung SM-G950F
   4 Samsung SM-G950U
   4 Samsung SM-N950U
   3 OnePlus ONEPLUS A5010
   3 Moto G (5
   2 SO-04H
   2 Generic Tablet
   2 WAS-LX2J
   2 Samsung SM-G930F
   2 SOV36
   2 KFSUWI
   2 moto g(6
   2 SO-01J
   2 Pixel 3
   2 moto x4
   2 Huawei ANE-LX1
   2 Samsung SM-N960U
   2 Pixel XL
   2 Moto G (5S
   2 D6603
   1 XiaoMi Redmi 4X
   1 OnePlus ONEPLUS A6003
   1 Asus Nexus 7
   1 Moto G Play
   1 Moto E (4
   1 Samsung SM-T535
   1 Asus X008D
   1 WAS-LX1A
   1 XT1069
   1 E6653
   1 Samsung SM-J320A
   1 Samsung SM-G955F
   1 Huawei CAM-L03
   1 Xperia Z4 Tablet Wifi
   1 SO-01K
   1 SOV32
   1 OnePlus ONEPLUS A6010
   1 SHV40
   1 SO-02J
   1 Asus Z017DA
   1 Nexus 6
   1 601SO
   1 G8142
   1 Samsung SM-A310F
   1 SH-01K
   1 SCH-I535
   1 SO-01H
   1 BBB100-6
   1 FTJ152C
   1 iPod
   1 Samsung SM-T285M
   1 XT1039
   1 d-

## Drop sessions table for privacy reasons

In [18]:
c.execute("DROP TABLE sessions")
c.execute("VACUUM")
conn.commit()
conn.close()